## This Notebook takes a quick crack at loading and training on the kaggle dataset that is similar to my project 

In [2]:
%matplotlib widget
import pandas as pd
import tensorflow as tf
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers
from sklearn import preprocessing
import matplotlib.pyplot as plt


In [44]:
import tensorflow as tf 
from tensorflow import keras
from keras import backend as K
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.layers.experimental import preprocessing

In [36]:
fname = '/Users/brad/Desktop/CS6620/Project/Data/solarenergy.csv' 
df = pd.read_csv(fname, sep=',', low_memory=False)
df.head()
df.dropna(axis=0, inplace=True)

In [37]:
y = df['solar_mw']
df.drop(['Datetime', 'solar_mw', 'wind-direction'], axis=1, inplace=True)
df.head()

,wind-speed,humidity,average-wind-speed-(period),average-pressure-(period),temperature
0,7.5,75.0,8.0,29.82,69.0
1,7.5,77.0,5.0,29.85,69.0
2,7.5,70.0,0.0,29.89,69.0
3,7.5,33.0,0.0,29.91,69.0
4,7.5,21.0,3.0,29.89,69.0


In [38]:
plt.figure()
plt.hist(y)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
df.isna().any()

wind-speed                     False
humidity                       False
average-wind-speed-(period)    False
average-pressure-(period)      False
temperature                    False
dtype: bool

In [40]:
# scale by largest value in each column 
for c in df.columns:
    cMax = df[c].max()
    print(f'Max for {c} is {cMax}')
    df[c] = df[c]/cMax

Max for wind-speed is 26.6
Max for humidity is 100.0
Max for average-wind-speed-(period) is 40.0
Max for average-pressure-(period) is 30.53
Max for temperature is 77.0


In [46]:
yMax = y.max()
y = y/yMax

In [23]:
df.columns

Index(['wind-direction', 'wind-speed', 'humidity',
       'average-wind-speed-(period)', 'average-pressure-(period)',
       'temperature'],
      dtype='object')

In [42]:
df.dtypes

wind-speed                     float64
humidity                       float64
average-wind-speed-(period)    float64
average-pressure-(period)      float64
temperature                    float64
dtype: object

In [59]:
def plot_loss(history):
  plt.figure()
  plt.plot(history.history['loss'], label='loss')
#   plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.grid(True)
  plt.show()
  plt.figure()
  plt.plot(history.history['coeff_determination'], label='R^2')
#   plt.plot(history.history['val_coeff_determination'], label='validation R^2')
  plt.xlabel('Epoch')
  plt.ylabel('R^2')
  plt.legend()
  plt.show()

In [41]:
# computes an R^2 (goodness of fit) approximation 
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [66]:
model = Sequential()
model.add(LSTM(128, input_shape=(5,1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))
# optimizer 
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-5)
# compile
model.compile(loss='mean_absolute_error', optimizer=opt)

In [47]:
x = df.to_numpy()
x.shape

(2206, 5)

In [49]:
y = y.to_numpy()
y.shape

(2206,)

In [54]:
x = x.reshape(-1, 5, 1)
y = y.reshape(-1, 1)
print(x.shape)

(2206, 5, 1)


In [67]:
history= model.fit(x=x, y=y, batch_size=10, epochs=100)
plot_loss(history)

Epoch 1/100
221/221 [==============================] - 3s 5ms/step - loss: 0.2378
Epoch 2/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1285
Epoch 3/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1208
Epoch 4/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1179
Epoch 5/100
221/221 [==============================] - 1s 4ms/step - loss: 0.1157
Epoch 6/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1142
Epoch 7/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1099
Epoch 8/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1145
Epoch 9/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1129
Epoch 10/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1157
Epoch 11/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1142
Epoch 12/100
221/221 [==============================] - 1s 5ms/step - loss: 0.1157
Epoch 13/100


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'coeff_determination'

In [69]:
preds = model.predict(x)

In [70]:
plt.figure()
plt.plot(y)
plt.plot(preds)
plt.legend(['actual','predicted'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …